<a href="https://colab.research.google.com/github/Solrak97/tf-vgg16-flowerdata/blob/main/VGG_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transferencia de aprendizaje utilizando VGG16

In [4]:
%%capture
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! rm -rf flowers
! kaggle datasets download alxmamaev/flowers-recognition
! unzip flowers-recognition.zip
! rm -r flowers-recognition.zip
! rm -rf flowers/sample_data/

In [5]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torch import nn
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from torch.utils.data import Subset

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Carga del dataset

In [6]:
transform_train = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
                                      transforms.ColorJitter(brightness=1, contrast=1, saturation=1),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ])


transform = transforms.Compose([transforms.Resize((224,224)),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                               ])

dataset = ImageFolder("flowers")
train, val = torch.utils.data.random_split(dataset, [4317 - 863, 863], generator=torch.Generator().manual_seed(42))

train.dataset.transform = transform_train
val.dataset.transform = transform

training_loader = torch.utils.data.DataLoader(train, batch_size=200, shuffle=True)
validation_loader = torch.utils.data.DataLoader(val, batch_size = 200, shuffle=False)

In [7]:
def im_convert(tensor):
  image = tensor.cpu().clone().detach().numpy()
  image = image.transpose(1, 2, 0)
  image = image * np.array((0.5, 0.5, 0.5)) + np.array((0.5, 0.5, 0.5))
  image = image.clip(0, 1)
  return image

## Modificación del modelo base VGG16

In [8]:
classes = ('margaritas', 'rosas', 'dientes de león', 'tulipanes', 'girasoles')

In [9]:
# Creación del modelo VGG16
model = models.vgg16(pretrained=True)

# Congelamiento del modelo base
for param in model.features.parameters():
  param.requires_grad = False

n_inputs = model.classifier[6].in_features
last_layer = nn.Linear(n_inputs, len(classes))

# Reemplzo de la capa de salida
model.classifier[6] = last_layer
model = model.to(device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

## Entrenamiento del modelo

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [11]:
epochs = 10
running_loss_history = []
running_corrects_history = []
val_running_loss_history = []
val_running_corrects_history = []

for e in range(epochs):
  
  running_loss = 0.0
  running_corrects = 0.0
  val_running_loss = 0.0
  val_running_corrects = 0.0
  
  for inputs, labels in training_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    _, preds = torch.max(outputs, 1)
    running_loss += loss.item()
    running_corrects += torch.sum(preds == labels.data)

  else:
    with torch.no_grad():
      for val_inputs, val_labels in validation_loader:
        val_inputs = val_inputs.to(device)
        val_labels = val_labels.to(device)
        val_outputs = model(val_inputs)
        val_loss = criterion(val_outputs, val_labels)

        _, val_preds = torch.max(val_outputs, 1)
        val_running_loss += val_loss.item()
        val_running_corrects += torch.sum(val_preds == val_labels.data)
      
    epoch_loss = running_loss/len(training_loader.dataset)
    epoch_acc = running_corrects.float()/ len(training_loader.dataset)
    running_loss_history.append(epoch_loss)
    running_corrects_history.append(epoch_acc)
    
    val_epoch_loss = val_running_loss/len(validation_loader.dataset)
    val_epoch_acc = val_running_corrects.float()/ len(validation_loader.dataset)
    val_running_loss_history.append(val_epoch_loss)
    val_running_corrects_history.append(val_epoch_acc)
    print('epoch :', (e+1))
    print('training loss: {:.4f}, acc {:.4f} '.format(epoch_loss, epoch_acc.item()))
    print('validation loss: {:.4f}, validation acc {:.4f} '.format(val_epoch_loss, val_epoch_acc.item()))

epoch : 1
training loss: 0.0039, acc 0.7122 
validation loss: 0.0027, validation acc 0.8320 
epoch : 2
training loss: 0.0013, acc 0.9085 
validation loss: 0.0022, validation acc 0.8563 
epoch : 3
training loss: 0.0006, acc 0.9658 
validation loss: 0.0021, validation acc 0.8737 
epoch : 4
training loss: 0.0003, acc 0.9887 
validation loss: 0.0023, validation acc 0.8633 
epoch : 5
training loss: 0.0001, acc 0.9939 
validation loss: 0.0023, validation acc 0.8667 
epoch : 6
training loss: 0.0001, acc 0.9980 
validation loss: 0.0024, validation acc 0.8725 
epoch : 7
training loss: 0.0000, acc 0.9988 
validation loss: 0.0027, validation acc 0.8633 
epoch : 8
training loss: 0.0000, acc 0.9986 
validation loss: 0.0025, validation acc 0.8783 
epoch : 9
training loss: 0.0000, acc 0.9997 
validation loss: 0.0026, validation acc 0.8714 
epoch : 10
training loss: 0.0000, acc 0.9988 
validation loss: 0.0028, validation acc 0.8737 


## Guardar modelo

In [15]:
torch.save(model.state_dict(), 'classifier.trc')